In [1]:
import os
from pathlib import Path
import logging

from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator, Descriptors, AllChem

In [2]:
dotenv_path = Path('..') / ".env"
if not dotenv_path.exists():
    raise FileNotFoundError(
        ".env file must be created in this folder with CSST_DB_USER, "
        + "CSST_DB_PASSWORD, CSST_DB_HOST, CSST_DB_PORT, and CSST_DB_NAME"
    )
load_dotenv(str(dotenv_path))

True

In [3]:
db_server = "postgresql+psycopg://{}:{}@{}:{}/{}".format(
    os.environ.get("CSST_DB_USER"),
    os.environ.get("CSST_DB_PASSWORD"),
    os.environ.get("CSST_DB_HOST"),
    int(os.environ.get("CSST_DB_PORT")),
    os.environ.get("CSST_DB_NAME"),
)

engine = create_engine(db_server)

In [4]:
# this query will get all experiments with a stir rate of 900 and
# a heating ramp rate of 0.5. It will only extract filtered processed data from the
# csst_reaction_processed_temperature_values table that is in the cooling state.
# It will remove all reactors that have a high uncertainty (>8) in the holding periods at 
# 10 and 60 C. It will also remove points with high individual uncertainties
query = '''SELECT distinct proc.average_temperature,
        proc.temperature_range,
        proc.average_transmission,
        proc.median_transmission,
        proc.transmission_std,
        proc.heating,
        proc.cooling,
        proc.holding,
        proc.filtered,
        proc.csst_reactor_id,
        reactor.conc,
        reactor.conc_unit,
        lpol.name as polymer_name,
        lpol.number_average_mw_min,
        lpol.number_average_mw_max,
        lsol.name as solvent_name,
        polymers.smiles as pol_smiles,
        solvents.smiles as sol_smiles
FROM csst_reactor_processed_temperature_values as proc
INNER JOIN csst_reactors AS reactor ON proc.csst_reactor_id = reactor.id
INNER JOIN lab_polymers AS lpol ON reactor.lab_pol_id = lpol.id
INNER JOIN lab_solvents AS lsol ON reactor.lab_sol_id = lsol.id
INNER JOIN polymers ON lpol.pol_id = polymers.id
INNER JOIN solvents ON lsol.sol_id = solvents.id
join csst_reactors cr on cr.id = proc.csst_reactor_id
join csst_experiments ce on cr.csst_experiment_id = ce.id
join csst_experiment_property_single_values cepsv on cepsv.csst_experiment_id = ce.id
join csst_temperature_programs ctp on ctp.id = cr.csst_temperature_program_id
where not ce.file_name like '%(10)%'
and cepsv.csst_property_id in (1, 7)
and cepsv.value = 900
and (ctp.experiment[1]->'rate'->'value')::text::numeric = 0.5
and proc.filtered = true
and proc.cooling = 1
and proc.csst_reactor_id not in (
    select distinct csst_reactor_id  from csst_reactor_processed_temperature_values proc
    where proc.holding = 1
    and ( proc.average_temperature = 10 or proc.average_temperature = 60 )
    and proc.transmission_std > 8
)
and proc.transmission_std <= 5
'''

In [5]:
# in order for this to work, you must first add all of the data to the database using the `add_data.py` script
with engine.connect() as conn:
    df = pd.read_sql(text(query), conn)

In [6]:
df

,average_temperature,temperature_range,average_transmission,median_transmission,transmission_std,heating,cooling,holding,filtered,csst_reactor_id,conc,conc_unit,polymer_name,number_average_mw_min,number_average_mw_max,solvent_name,pol_smiles,sol_smiles
0,10.0,1.0,0.0,0.0,0.000000e+00,0,1,0,True,7,15.29,mg/ml,Polypropylene,15000.0,15000.0,Ethanol,[*]CC([*])C,CCO
1,10.0,1.0,0.0,0.0,0.000000e+00,0,1,0,True,26,15.00,mg/ml,Polypropylene,15000.0,15000.0,Diethyl ether,[*]CC([*])C,CCOCC
2,10.0,1.0,0.0,0.0,0.000000e+00,0,1,0,True,27,30.50,mg/ml,Polypropylene,15000.0,15000.0,Diethyl ether,[*]CC([*])C,CCOCC
3,10.0,1.0,0.0,0.0,0.000000e+00,0,1,0,True,28,49.80,mg/ml,Polypropylene,15000.0,15000.0,Diethyl ether,[*]CC([*])C,CCOCC
4,10.0,1.0,0.0,0.0,0.000000e+00,0,1,0,True,31,29.26,mg/ml,Polypropylene,15000.0,15000.0,Acetonitrile,[*]CC([*])C,CC#N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50491,60.0,1.0,100.0,100.0,1.421085e-14,0,1,0,True,160,15.06,mg/ml,Poly(chlorotrifluoroethylene),NaN,NaN,Diethyl ether,[*]C(F)(F)C([*])(F)Cl,CCOCC
50492,60.0,1.0,100.0,100.0,1.421085e-14,0,1,0,True,161,15.24,mg/ml,Poly(chlorotrifluoroethylene),NaN,NaN,Benzene,[*]C(F)(F)C([*])(F)Cl,C1=CC=CC=C1
50493,60.0,1.0,100.0,100.0,0.000000e+00,0,1,0,True,238,15.10,mg/ml,mPEG,5000.0,5000.0,Benzyl alcohol,[*]CCO[*],c1ccc(cc1)CO
50494,60.0,1.0,100.0,100.0,0.000000e+00,0,1,0,True,1034,15.28,mg/ml,Polyvinylpyrrolidone,10000.0,10000.0,"2, 4, 6- trimethylpyridine",[*]CC([*])N1CCCC1=O,CC1=CC(=NC(=C1)C)C


In [7]:
def one_hot_encode(df, encoding_col: str = 'polymer_name', mat: str = 'pol'):
    data = []
    cols = [col for col in df.columns if 'fp_' not in col]
    values = df[encoding_col].unique()
    for index, row in df.iterrows():
        r = {col: row[col] for col in cols}
        for value in values:
            if row[encoding_col] == value:
                r[f"fp_{mat}_{value}"] = 1
            else:
                r[f"fp_{mat}_{value}"] = 0
        data.append(r.copy())
    df = pd.DataFrame(data)
    df = df.fillna(0)
    return df

In [8]:
def fp(smiles: str, mat: str = 'sol'):
    """Generating rdkit fingerprint from a variety of descriptors available from rdkit

    Args:
        mat: what the name is in fp_{mat}. Could be sol or pol (but you may not fingerprint
            polymer as it is a small polymer dataset and Im not sure how these descriptors
            would do on it)
    """
    m = Chem.MolFromSmiles(smiles)
    # can use various types of fingerprints
    rdkit_descriptors = {}
    # rdkit_descriptors.update({f"fp_{mat}_{k}": v for k, v in Descriptors.CalcMolDescriptors(m).items()})
    morgan_generator = rdFingerprintGenerator.GetMorganGenerator(radius=2)
    morgan_fp = morgan_generator.GetFingerprint(m)
    rdkit_descriptors.update({f"fp_{mat}_morgan_bit_{index}": int(bit) for index, bit in enumerate(morgan_fp.ToBitString())})
    return rdkit_descriptors

def fp_df(df, smiles_col: str = 'sol_smiles', mat: str = 'sol'):
    data = []
    cols = [col for col in df.columns if 'fp_' not in col]
    for index, row in df.iterrows():
        r = {col: row[col] for col in cols}
        mat_fp = fp(row[smiles_col], mat)
        r.update(mat_fp)
        data.append(r.copy())
    df = pd.DataFrame(data)
    df.fillna(0)
    return df

In [9]:
pols = df[['pol_smiles', 'polymer_name']].drop_duplicates()
sols = df[['sol_smiles', 'solvent_name']].drop_duplicates()

In [10]:
pols = one_hot_encode(pols)

In [11]:
sols = fp_df(sols)

In [12]:
df = df.merge(pols, on='polymer_name')
df = df.merge(sols, on='solvent_name')

In [13]:
df

,average_temperature,temperature_range,average_transmission,median_transmission,transmission_std,heating,cooling,holding,filtered,csst_reactor_id,...,fp_sol_morgan_bit_2038,fp_sol_morgan_bit_2039,fp_sol_morgan_bit_2040,fp_sol_morgan_bit_2041,fp_sol_morgan_bit_2042,fp_sol_morgan_bit_2043,fp_sol_morgan_bit_2044,fp_sol_morgan_bit_2045,fp_sol_morgan_bit_2046,fp_sol_morgan_bit_2047
0,10.0,1.0,0.0,0.0,0.000000e+00,0,1,0,True,7,...,0,0,0,0,0,0,0,0,0,0
1,11.0,1.0,0.0,0.0,0.000000e+00,0,1,0,True,7,...,0,0,0,0,0,0,0,0,0,0
2,12.0,1.0,0.0,0.0,0.000000e+00,0,1,0,True,7,...,0,0,0,0,0,0,0,0,0,0
3,13.0,1.0,0.0,0.0,0.000000e+00,0,1,0,True,7,...,0,0,0,0,0,0,0,0,0,0
4,14.0,1.0,0.0,0.0,0.000000e+00,0,1,0,True,7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50491,56.0,1.0,100.0,100.0,2.842171e-14,0,1,0,True,274,...,0,0,0,0,0,0,0,0,0,0
50492,57.0,1.0,100.0,100.0,2.842171e-14,0,1,0,True,274,...,0,0,0,0,0,0,0,0,0,0
50493,58.0,1.0,100.0,100.0,2.842171e-14,0,1,0,True,274,...,0,0,0,0,0,0,0,0,0,0
50494,59.0,1.0,100.0,100.0,2.842171e-14,0,1,0,True,274,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Can use this to extract fingerprint columns for machine learning
fp_cols = [col for col in df.columns if 'fp_' in col] + ['average_temperature', 'conc']
y_col = 'average_transmission'

In [15]:
df[fp_cols]

,fp_pol_Polypropylene,fp_pol_Poly (vinylidene fluoride),fp_pol_PEO,fp_pol_Nylon-12,fp_pol_mPEG,fp_pol_Poly(chlorotrifluoroethylene),fp_pol_Poly4vinylphenol,fp_pol_Polyethylene terephthalate,fp_pol_PEG,fp_pol_Poly ethylene,...,fp_sol_morgan_bit_2040,fp_sol_morgan_bit_2041,fp_sol_morgan_bit_2042,fp_sol_morgan_bit_2043,fp_sol_morgan_bit_2044,fp_sol_morgan_bit_2045,fp_sol_morgan_bit_2046,fp_sol_morgan_bit_2047,average_temperature,conc
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10.0,15.29
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11.0,15.29
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,12.0,15.29
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,13.0,15.29
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,14.0,15.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50491,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,56.0,15.10
50492,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,57.0,15.10
50493,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,58.0,15.10
50494,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,59.0,15.10
